# Section 1004.1.2 - Areas without fixed seating

## Building Code

```
The number of occupants shall be computed at the rate of one occupant per unit of area as prescribed in Table 1004.1.2. For areas without fixed seating, the occupant load shall be not less than that number determined by dividing the floor area under consideration by the occupant load factor assigned to the function of the space as set forth in Table 1004.1.2. Where an intended function is not listed in Table 1004.1.2, the building official shall establish a function based on a listed function that most nearly resembles the intended function. 

Exception: Where approved by the building official, the actual number of occupants for whom each occupied space, floor or building is designed, although less than those determined by calculation, shall be permitted to be used in the determination of the design occupant load. 
```

### TABLE 1004.1.2 MAXIMUM FLOOR AREA ALLOWANCES PER OCCUPANT 

In [19]:
import json
import pandas as pd
from math import ceil
# Load the table from the JSON file
with open('table_1004_1_2.json', 'r') as f:
    table_1004_1_2 = json.load(f)

In [2]:
# Define the SPARQL endpoint URL and query file path
endpoint_url = "http://localhost:3030/SmartReviewHotel/sparql"

In [3]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper(endpoint_url)
with open("..\..\..\SPARQL\\roomOccupantLoad.sparql", "r") as file:
    sparql_query = file.read()

# Set the SPARQL query and the return format
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)

# Execute the query and get the results in JSON format
results = sparql.query().convert()

In [20]:
# Process and organize the results by level
data_by_level = {}

for result in results["results"]["bindings"]:
    level = result["level"]["value"]
    room_id = result["label"]["value"]
    name = result["name"]["value"]
    function = result["function"]["value"]
    area = float(result["areaValue"]["value"])

    # Lookup occupant load factor based on function
    occupant_load_factor_str = table_1004_1_2.get(function, "0")
    try:
        occupant_load_factor = float(occupant_load_factor_str)
    except ValueError:
        occupant_load_factor = 0  # If occupant load factor is not a number, default to 0

    if occupant_load_factor == 0:
        reason = f"No occupant load factor found for function '{function}'. Calculation skipped."
        gross_occupant_load = "N/A"
    else:
        gross_occupant_load = ceil(area / occupant_load_factor)
        reason = (
            f"The room has a Function of Space of {function} and room area of {area}. "
            f"The Occupant Load Factor associated with the Function of Space is {occupant_load_factor}. "
            f"This resulted in a gross Occupant Load of {gross_occupant_load}."
        )

    # Append room data to the level's data list
    if level not in data_by_level:
        data_by_level[level] = []
    data_by_level[level].append({
        "Room ID": room_id,
        "Name": name,
        "Reason": reason
    })

# Generate tables for each level and display as DataFrames
for level, rooms in data_by_level.items():
    df = pd.DataFrame(rooms)
    print(f"Level {level} Room Details:")
    display(df.style.set_table_attributes("style='display:inline-block; max-height:300px; overflow-y:scroll;'"))

Level 01-GROUND FLOOR Room Details:


,Room ID,Name,Reason
0,495,King Suite,The room has a Function of Space of Residential and room area of 344.04253472221376. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 2.
1,494,King Suite,The room has a Function of Space of Residential and room area of 344.04253472220483. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 2.
2,497,1 Bedroom Suite - ADA,The room has a Function of Space of Residential and room area of 267.99999999997726. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 2.
3,496,1 Bedroom Suite - ADA,The room has a Function of Space of Residential and room area of 268.00000000001774. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 2.
4,489,King Suite,The room has a Function of Space of Residential and room area of 343.50694444447686. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 2.
5,488,King Suite,The room has a Function of Space of Residential and room area of 343.5069444444259. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 2.
6,563,Outdoor Storage,The room has a Function of Space of Accessory Storage Area and room area of 67.46918402775646. The Occupant Load Factor associated with the Function of Space is 300.0. This resulted in a gross Occupant Load of 1.
7,560,Stair #2,The room has a Function of Space of Residential and room area of 253.46592881942456. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 2.
8,559,Outdoor Storage,The room has a Function of Space of Accessory Storage Area and room area of 45.19257090874197. The Occupant Load Factor associated with the Function of Space is 300.0. This resulted in a gross Occupant Load of 1.
9,562,Maintenance,The room has a Function of Space of Accessory Storage Area and room area of 158.3656684027319. The Occupant Load Factor associated with the Function of Space is 300.0. This resulted in a gross Occupant Load of 1.


Level 03-THIRD FLOOR Room Details:


,Room ID,Name,Reason
0,570,Elevator 3-1,The room has a Function of Space of Residential and room area of 60.858206548943116. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 1.
1,571,Elevator 3-2,The room has a Function of Space of Residential and room area of 60.85820654894306. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 1.
2,316,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 427.2534722222744. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
3,320,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 426.7673611111469. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
4,319,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 426.76736111107647. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
5,318,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 427.2534722222556. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
6,317,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 427.25347222218244. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
7,324,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 427.5703125000174. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
8,323,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 427.5703124999702. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
9,322,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 427.01041666670386. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.


Level 02-SECOND FLOOR Room Details:


,Room ID,Name,Reason
0,568,Elevator 2-1,The room has a Function of Space of Residential and room area of 60.858206548943116. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 1.
1,569,Elevator2-2,The room has a Function of Space of Residential and room area of 60.85820654894303. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 1.
2,290,Elevator Lobby,The room has a Function of Space of Residential and room area of 165.43625769918324. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 1.
3,289,Vestibule,The room has a Function of Space of Residential and room area of 122.81423611103973. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 1.
4,288,Chute Room,The room has a Function of Space of Residential and room area of 90.82986111120243. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 1.
5,287,Storage,The room has a Function of Space of Accessory Storage Area and room area of 237.12586805565988. The Occupant Load Factor associated with the Function of Space is 300.0. This resulted in a gross Occupant Load of 1.
6,294,Corridor,The room has a Function of Space of Residential and room area of 901.2892331506663. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 5.
7,221,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 427.0104166666489. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
8,220,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 426.76736111114695. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
9,219,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 426.76736111109335. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.


Level 04-FOURTH FLOOR Room Details:


,Room ID,Name,Reason
0,416,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 427.2534722222744. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
1,420,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 426.7673611111469. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
2,419,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 426.76736111107647. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
3,418,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 427.2534722222555. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
4,417,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 427.25347222218244. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
5,424,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 427.5703125000174. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
6,423,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 427.5703124999702. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
7,422,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 427.01041666670386. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
8,421,Queen/Queen Suite,The room has a Function of Space of Residential and room area of 427.0104166666376. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 3.
9,428,King Suite,The room has a Function of Space of Residential and room area of 343.2638888888886. The Occupant Load Factor associated with the Function of Space is 200.0. This resulted in a gross Occupant Load of 2.
